In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/5.src'):
    shutil.rmtree('./file/5.src')

os.makedirs('./file/5.src')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/4.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['Product No'] = input_['Product No'].astype(int)
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1

                    try:
                        request_url = f'''https://gates.scene7.com/is/image/{input_.loc[a, 'Asset']}?req=set,json,UTF-8&labelkey=label&handler=s7classics7sdkJSONResponse'''
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                
                        if resp.status_code == 200:
                            if 'Error while processing Fvctx image' in resp.text and 'Error while processing Fvctx image' in resp.text:
                                break
                                
                            dict_ = json.loads(resp.text.split('s7classics7sdkJSONResponse(')[1].split(',"");')[0])
                            if dict_['set']['n'].strip() == input_.loc[a, 'Asset']:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                dict_src = {}
                if 'Error while processing Fvctx image' in resp.text and 'Error while processing Fvctx image' in resp.text:
                    pass
                else:
                    list_src = [dict_['s']['n'].strip() if 's' in dict_ else dict_['set']['n'] for dict_ in dict_['set']['item']] if type(dict_['set']['item']) == list else [dict_['set']['item']['s']['n'].strip() if 's' in dict_['set']['item'] else dict_['set']['item']['i']['n'].strip()]
                    for i, src in enumerate(list_src):
                        dict_src[str(i)] = f'''https://gates.scene7.com/is/image/{src}'''

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'Product No': input_.loc[a, 'Product No'],
                                         'Product Name': input_.loc[a, 'Product Name'],
                                         'Product Number': input_.loc[a, 'Product Number'],
                                         'Country': input_.loc[a, 'Country'],
                                         'No': input_.loc[a, 'No'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Type': input_.loc[a, 'Type'],
                                         'SKU': input_.loc[a, 'SKU'],
                                         'UPC': input_.loc[a, 'UPC'],
                                         'Title': input_.loc[a, 'Title'],
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Information': input_.loc[a, 'Json_Information'],
                                         'Json_Kit': input_.loc[a, 'Json_Kit'],
                                         'Part Code': input_.loc[a, 'Part Code'],
                                         'Asset': input_.loc[a, 'Asset']}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Product No']}.{input_.loc[a, 'Product Name']} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(50):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Product No', 'No'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/5.src/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Product No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-src_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/5.src'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/5.src/{file}',
              f'''./file/5.src/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：1] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 195.
[剩余数量：1000] - [当前时间：18:23:47]

[状态：ok，尝试次数：15] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 167.
[剩余数量：999] - [当前时间：18:23:47]

[状态：ok，尝试次数：1] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 186.
[剩余数量：998] - [当前时间：18:23:47]

[状态：ok，尝试次数：16] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 115.
[剩余数量：997] - [当前时间：18:23:47]

[状态：ok，尝试次数：5] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 194.
[剩余数量：996] - [当前时间：18:23:48]

[状态：ok，尝试次数：3] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 189.
[剩余数量：995] - [当前时间：18:23:48]

[状态：ok，尝试次数：11] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 133.
[剩余数量：994] - [当前时间：18:23:48]

[状态：ok，尝试次数：5] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 196.
[剩余数量：993] - [当前时间：18:23:48]

[状态：ok，尝试次数：12] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 184.
[剩余数量：992] - [当前时间：18:23:48]

[状态：ok，尝试次数：10] - 100.00% - crawler_3 > 98.Machinery Spare Parts > 193.
[剩余数量：991] - [

Progress: 100%|██████████████████████████████| 3/3 [00:00<00:00, 3859.79it/s]

Done ~
